1. IMPORT LIBRARY YANG DIPERLUKAN

In [1]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter, find_peaks
import matplotlib.pyplot as plt

2. MEMUAT DATASET CSV DAN MENAMPILKAN SEMUA BARIS DATA

In [ ]:
file_path = 'E:/SKRIPSI/Dataset/data uji/Reza/sesi 4/gsr_data.csv'  # Ganti dengan path file CSV Anda
data = pd.read_csv(file_path)

# Pastikan kolom 'Timestamp' dan 'Conductance (µS)' ada
required_columns = ['Timestamp', 'Conductance (µS)']
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Kolom '{col}' tidak ditemukan dalam data CSV.")

# Ubah 'Timestamp' menjadi tipe datetime jika diperlukan
try:
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])
except Exception as e:
    print(f"Timestamp tidak dalam format datetime: {e}. Melanjutkan dengan format asli.")

# Menampilkan informasi dataset
print("\nRingkasan data:")
print(data.info())

3. FILTERING NOISE PADA SINYAL MENGGUNAKAN SAVITZKY-GOLAY-FILTER

In [ ]:
def apply_savgol_filter(data, window_length, polyorder):
    # Validasi panjang jendela
    if window_length > len(data):
        raise ValueError(f"window_length ({window_length}) harus lebih kecil atau sama dengan panjang data ({len(data)}).")
    if window_length % 2 == 0:
        window_length += 1  # Pastikan ganjil
    return savgol_filter(data, window_length=window_length, polyorder=polyorder)

# Parameter Savitzky-Golay Filter
window_length = min(51, len(data))  # Sesuaikan jika data terlalu pendek
if window_length % 2 == 0:
    window_length -= 1
polyorder = 3  # Orde polinomial untuk fitting

# Terapkan Savitzky-Golay Filter
data['Filtered_Conductance'] = apply_savgol_filter(data['Conductance (µS)'], window_length, polyorder)

# Plot hasil filtering
plt.figure(figsize=(12, 6))
plt.plot(data['Timestamp'], data['Conductance (µS)'], label='Original')
plt.plot(data['Timestamp'], data['Filtered_Conductance'], label='Filtered (Savitzky-Golay)', color='red')
plt.xlabel('Timestamp')
plt.ylabel('Conductance (µS)')
plt.legend()
plt.title('GSR Signal Before and After Savitzky-Golay Filtering')
plt.show()

4. DETEKSI PUNCAK 

In [ ]:
# Deteksi puncak SCR
peaks, _ = find_peaks(data['Filtered_Conductance'], height=0.05)  # Sesuaikan threshold
plt.figure(figsize=(12, 6))
plt.plot(data['Timestamp'], data['Filtered_Conductance'], label='Filtered')
plt.plot(data['Timestamp'].iloc[peaks], data['Filtered_Conductance'].iloc[peaks], 'x', label='Peaks')
plt.xlabel('Timestamp')
plt.ylabel('Filtered Conductance (µS)')
plt.title('Detected SCR Peaks')
plt.legend()
plt.show()

5. LABELLING

In [ ]:
# Gunakan nama file untuk menentukan sesi
#if 'sesi 1' in file_path.lower():
#    label = 'Normal'  # Tidak stres
#elif 'sesi 2' in file_path.lower():
#    label = 'Stres Ringan'  # Stres ringan
#elif 'sesi 3' in file_path.lower():
#    label = 'Stres Sedang'  # Stres sedang
#elif 'sesi 4' in file_path.lower():
#    label = 'Stres Berat'  # Stres tinggi
#else:
#    raise ValueError("Sesi tidak dikenali dari nama file.")

# Tambahkan kolom label ke data
#data['Label'] = label

6. MENYIMPAN CSV DATA YANG SUDAH DI PREPROCESSING

In [ ]:
output_file_path = file_path.replace('.csv', '_processed.csv')
data.to_csv(output_file_path, index=False)

print(f"File CSV baru dengan label telah disimpan di: {output_file_path}")